<a href="https://colab.research.google.com/github/CeHaga-UFRJ/bmt-ufrj-2024.P1/blob/main/BMT_Trab1_Carlos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import nltk
from nltk.tokenize import word_tokenize
from lxml import etree

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
!git clone https://github.com/CeHaga-UFRJ/bmt-ufrj-2024.P1.git repo
!mv repo/data data
!mv repo/config config

fatal: destination path 'repo' already exists and is not an empty directory.


In [35]:
class Indexer:
  def __init__(self, cfg_file):
    self.inverted_list = {}
    self.xml_root = []
    self.output_file = ""
    self._read_cfg(cfg_file)

    # for i in range(len(base)):
    #   tokens = word_tokenize(base[i].lower())
    #   for token in tokens:
    #     docs = self.inverted_list.get(token, [])
    #     docs.append(i+1)
    #     self.inverted_list[token] = docs

  def _read_cfg(self, cfg_file):
    with open(cfg_file, 'r') as config:
      for line in config:
        command, file_name = line.replace("\n","").split("=")
        if(command == 'LEIA'):
          root = self._get_xml_root(file_name)
          self.xml_root.append(root)
        elif(command == 'ESCREVA'):
          self.output_file = file_name

  def __str__(self):
    keys = [i for i in self.inverted_list.keys()]
    keys.sort()

    index_text = ""
    for key in keys:
      index_text += key + " => " + str(self.inverted_list[key]) + "\n"

    return index_text

  def _get_xml_root(self, file_name):
    tree = etree.parse(file_name)
    return tree.getroot()

  def _get_abstract_from_record(self, record):
    abstract_element = record.find('ABSTRACT')
    if abstract_element is not None:
      return abstract_element.text.strip()

    abstract_element = record.find('EXTRACT')
    if abstract_element is not None:
      return abstract_element.text.strip()

    return None

In [36]:
Indexer('config/gli.cfg')